In [1]:
# json library is used to handle json files, here, it is the data source retrieved by the perceval module.
import json 
# to write and read csv files, to show the output in the end
import csv  

# importing pandas
# pandas is used for handling huge data using dataframe
import pandas as pd

# to handle the time formats, like to determine 'created_at' of an issue or pr.
from datetime import datetime, date, timedelta

# dictionaries are a convenient way to store data for later retrieval by name (key).
from collections import defaultdict  

# it is used to send http requests, I used to get the year in which the project created to do the analysis, using requests and github api.
import requests 

from prettytable import from_csv

In [2]:
repos = ['badgeyay','susi_android']

ctypes = ('commit','pull_request','issue')

In [3]:
# function to get the required details of commits
# commit has a different json structure unlike issue/pr

def details_commit(commit):
    # load the commit data into the object
    data = commit['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the commit
            'hash': data['commit'],
            # get the author_name
            'author': data['Author'],  
            # get the date at which the commit was created
            'created_date': datetime.strptime(data['CommitDate'],"%a %b %d %H:%M:%S %Y %z")  
    }
    return content

In [4]:
# function to get the required details of issue/pull requests
# as issue/pr has the same json structure in the data source scraped by perceval
# I wrote a single function to get the either issue/pr details 

def details_ipr(item):
    # load the commit data into the object
    data = item['data']
    # traverse through the json line to find the required data
    content ={
            # get the hash of the issue/pr
            'hash': data['id'],
            # get the author_name
            'author': data['user']['login'],  
            # get the date at which the issue/pr was created
            'created_date': datetime.strptime(data['created_at'],"%Y-%m-%dT%H:%M:%SZ")  
    }
    return content 

In [5]:
def getContents(repo):
    # contents is to store the details of each contribution whether it is a commit, issue or pr.
    # using a defaultdict of list so that I can store the sorted details according to the ctype as (key, value) 
    contents = defaultdict(list)

    # to filter out commit, issue, pr details from the data source and store them seperately in dict.
    # loading the file into an object
    with open('../data/%s.json'%repo) as datasrc:
        for line in datasrc:
            # load the line in the json format so as to iterate to get the required results
            line = json.loads(line)
            # if it is a commit, get the details of commit
            if line['category'] == 'commit':    
                content = details_commit(line) 
            # if it is a issue, get the details of issue
            elif line['category'] == 'issue':    
                content = details_ipr(line)
            # if it is a pr, get the details of pr
            elif line['category'] == 'pull_request':    
                content = details_ipr(line) 
            # add the (key, value) to the list
            contents[line['category']].append(content)
        return contents

In [6]:
initial_date = datetime.combine(date.today() - timedelta(3*365/12), datetime.min.time())   # convert datetime.date to datetime.datetime

In [7]:
repodata = defaultdict(list)

for repo in repos:
    repocontents = getContents(repo)
    total = 0
    for ctype in ctypes:
        count = 0
        for item in repocontents[ctype]:
            if item['created_date'].replace(tzinfo=None) >= initial_date:
                count += 1
        total += count
        repodata[ctype].append(count)
    repodata['total'].append(total)

In [8]:
print("Repositories Details in the past three months\n")
for item in dict(repodata):
    # print the total activity quaterly
    print (item, dict(repodata)[item])  

Repositories Details in the past three months

commit [102, 78]
pull_request [64, 140]
issue [0, 262]
total [166, 480]


In [9]:
# add headers to the csv file 
header = ['Repository','# Commits','# PullRequests','# Issues','# Total']
# opening a new csv to write the data into it.
with open('result.csv', 'w') as file:
    # intilize the writer object
    writer = csv.writer(file)
    # wring the header first
    writer.writerow(header)
    # to map the similar index of multiple containers so that they can be added in single entity i.e, rows
    rows = zip(repos,repodata['commit'],repodata['pull_request'],repodata['issue'],repodata['total'])
    # writing all the rows at a time
    writer.writerows(rows)

In [10]:
# to show the output in the form of a table
# load the csv file into a object
with open("result.csv", "r") as csvfile: 
    # using from_csv method from prettytable module
    csvtable = from_csv(csvfile)
    
# print the prettified table
print(csvtable)

+--------------+-----------+----------------+----------+---------+
|  Repository  | # Commits | # PullRequests | # Issues | # Total |
+--------------+-----------+----------------+----------+---------+
|   badgeyay   |    102    |       64       |    0     |   166   |
| susi_android |     78    |      140       |   262    |   480   |
+--------------+-----------+----------------+----------+---------+
